In [3]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pdb
import requests
import re

import numpy as np
import pandas as pd
from functools import reduce
from collections import Counter

import networkx as nx

import signal

import warnings
warnings.filterwarnings("ignore")

from wiki_intro_scrapper import WikiIntroScrapper
from WikiMultiQuery import wiki_multi_query
from graph_helpers import create_dispersion_df, sort_dict_values, format_categories, compare_categories, rank_order

%aimport wiki_intro_scrapper
%aimport WikiMultiQuery
%aimport graph_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
class GraphCreator:

    def __init__(self, entry):
        self.graph = nx.DiGraph()

        self.entry = entry

        wis = WikiIntroScrapper(f"https://en.wikipedia.org/wiki/{entry}")
        wis.parse_intro_links()

        self.intro_nodes = wis.intro_link_titles

        self.visited = {entry}
        self.next_links = []
        
        self.categories = {}
        
        self.redirect_targets = []
        self.redirect_sources = {}
        
        self.query_articles([entry])

        # setup timeout function

        def handle_alarm(signum, frame):
            raise RuntimeError

        signal.signal(signal.SIGALRM, handle_alarm)

    def add_edges(self, articles):
        for article in articles:
            
            self.categories[article['title']] = format_categories([category.split("Category:")[1] for category in article['categories'] if not bool(re.findall(r"(articles)|(uses)|(commons)|(category\:use)", category, re.I))])
            
            self.graph.add_edges_from(
                [(article['title'], link) for link in article['links']])
            self.graph.add_edges_from(
                [(linkhere, article['title']) for linkhere in article['linkshere']])

    def update_edge_weights(self):
        for edge in self.graph.out_edges:
            weight = compare_categories(edge[0], edge[1], self.categories)
            self.graph.add_edge(edge[0], edge[1], weight=weight)
            
        for edge in self.graph.in_edges:
            weight = compare_categories(edge[0], edge[1], self.categories)
            self.graph.add_edge(edge[0], edge[1], weight=weight)
    
    def get_edge_weights(self):
        edge_weights = []
        for edge in self.graph.edges:
            edge_weights.append((edge[0], edge[1], self.graph.get_edge_data(edge[0], edge[1])['weight']))
        
#         for edge in self.graph.in_edges:
#             edge_weights.append((edge[0], edge[1], self.graph.get_edge_data(edge[0], edge[1])['weight']))
        
        return pd.DataFrame(edge_weights, columns=["source_node", "target_node", "edge_weight"]).sort_values("edge_weight", ascending=False).reset_index().drop("index", axis=1)
    
    def plot_graph(self):
        nx.draw(self.graph)
        plt.show()

    def get_shared_categories_with_source(self):
        cat_matches = {}
        for node in self.graph.nodes:
            cat_matches[node] = compare_categories(self.entry, node, self.categories, starting_count=0)
        return sort_dict_values(cat_matches, ['node', 'category_matches_with_source'], 'category_matches_with_source', ascending=False)
            
            
    def get_degrees(self):
        return sort_dict_values(dict(self.graph.degree()), ["node", "degree"], "degree",)

    def get_edges(self):
        in_edges = sort_dict_values(dict(Counter([edge[1] for edge in self.graph.in_edges()])), 
                            ['node', 'in_edges'], "in_edges")
        out_edges = sort_dict_values(dict(Counter([edge[0] for edge in self.graph.out_edges()])), 
                            ["node", 'out_edges'], 'out_edges')

        return in_edges.merge(out_edges, on="node")
    
    def get_centrality(self):
        return sort_dict_values(nx.eigenvector_centrality(self.graph, weight="weight"), ["node", "centrality"], "centrality")

    def get_dispersion(self, comparison_node=None, max_nodes=25_000):
        if not comparison_node:
            comparison_node = self.entry
            
        if max_nodes is None or len(self.graph.nodes) <= max_nodes:
            print("FULL")
            return sort_dict_values(nx.dispersion(self.graph, u=comparison_node), ['node', 'dispersion'], 'dispersion')
        else:
            print("EGO")
            # if the network is too large, perform calculation on ego graph of entry node
            ego = self.create_ego()
            return sort_dict_values(nx.dispersion(ego, u=comparison_node), ['node', 'dispersion'], 'dispersion')

    def get_pageranks(self):
        page_ranks = sorted([(key, value) for key, value in nx.algorithms.link_analysis.pagerank(
            self.graph, weight='weight').items()], key=lambda x: x[1], reverse=True)
        return pd.DataFrame(page_ranks, columns=["node", "page_rank"])

    def get_reciprocity(self):
        return sort_dict_values(nx.algorithms.reciprocity(self.graph, self.graph.nodes), ['node', 'reciprocity'], 'reciprocity')

    def get_adjusted_reciprocity(self):
        r = self.get_reciprocity()
        d = self.get_degrees()

        r_d = r.merge(d, on="node", how="inner")
        r_d['adjusted_reciprocity'] = r_d.reciprocity * r_d.degree

        adjusted_reci = r_d.sort_values("adjusted_reciprocity", ascending=False)
        return adjusted_reci.reset_index().drop(["degree", "reciprocity", "index"], axis=1)
    
    def get_shortes_path(self, source=None, ascending=False):
        if not source:
            source = self.entry
            
        paths = nx.algorithms.single_source_shortest_path_length(self.graph, source)
        return sort_dict_values(paths, ["node", "shortest_path_length_from_source"], "shortest_path_length_from_source", ascending=ascending)
    
    def get_dominator_counts(self, source=None):
        if not source:
            source = self.entry
            
        dom_dict = nx.algorithms.dominance.immediate_dominators(self.graph, start=source)
        
        dom_counts = {}

        for key, value in dom_dict.items():
            if value in dom_counts:
                dom_counts[value] += 1
            else:
                dom_counts[value] = 1
        for node in self.graph.nodes:
            if not node in dom_counts:
                dom_counts[node] = 0
        
        return sort_dict_values(dom_counts, ['node', 'immediate_dominator_count'], 'immediate_dominator_count')
    
    def get_hits(self):
        hits = nx.algorithms.link_analysis.hits_alg.hits(self.graph, max_iter=1000)
        return (sort_dict_values(hits[1], ['node', 'hits_authority'], 'hits_authority')
                .merge(sort_dict_values(hits[0], ['node', 'hits_hub'], 'hits_hub'), on="node"))
    
    def get_features_df(self, rank=False):
        dfs = []
        if rank:
            dfs.append(rank_order(self.get_degrees(), 'degree', ascending=False))
            dfs.append(rank_order(self.get_shared_categories_with_source(), 'category_matches_with_source', ascending=False))
            dfs.append(self.get_edges())
            dfs.append(rank_order(self.get_centrality(), 'centrality', ascending=True))
            dfs.append(rank_order(self.get_dispersion(), "dispersion", ascending=True))
            dfs.append(rank_order(self.get_pageranks(), "page_rank", ascending=False))
            dfs.append(rank_order(self.get_adjusted_reciprocity(), "adjusted_reciprocity", ascending=False))
            dfs.append(rank_order(self.get_shortes_path(), "shortest_path_length_from_source", ascending=True))
        
        else:
            dfs.append(self.get_degrees())
            dfs.append(self.get_shared_categories_with_source())
            dfs.append(self.get_edges())
            dfs.append(self.get_centrality())
            dfs.append(self.get_dispersion())
            dfs.append(self.get_pageranks())
            dfs.append(self.get_adjusted_reciprocity())
            dfs.append(self.get_shortes_path())
        
        return reduce(lambda left, right: pd.merge(left, right, on="node", how="outer"), dfs)
        
    
    def create_ego(self, node=None):
        if not node:
            node = self.entry

        ego = nx.ego_graph(self.graph, node)
        ego.name = node
        return ego

    def expand_network(self, group_size=10, timeout=10, log_progress=False):

        num_links = len(self.next_links)

        link_group = []

        for i in range(num_links):
            link = self.next_links.pop(0)
            if not link in self.visited:

                link_group.append(link)

                if len(link_group) == group_size or (i == num_links - 1 and len(link_group) > 0):
                    print("{:.2%}".format(i/num_links)) if log_progress else None
                    try:
                        signal.alarm(timeout)
                        self.visited.update(link_group)
                        self.query_articles(link_group)
                        signal.alarm(0)
                        link_group = []
                    except:
                        link_group = []
                        continue
        signal.alarm(0)

    def update_redirects(self, articles):
        for article in articles:
            if article.get("redirects"):
                self.redirect_targets.append(article["title"])
                for redirect in article["redirects"]:
                    self.redirect_sources[redirect] = len(self.redirect_targets) - 1
    
    def redraw_redirects(self):
        edges = list(self.graph.edges) # need this copy so 'edges' doesn't change size on iteration
        for edge in edges:
            if edge[0] in self.redirect_sources:
                self.graph.add_edge(self.redirect_targets[self.redirect_sources[edge[0]]], edge[1])
                
            if edge[1] in self.redirect_sources:
                self.graph.add_edge(edge[0], self.redirect_targets[self.redirect_sources[edge[1]]])
        
        self.remove_redirect_nodes()
    
    def remove_redirect_nodes(self):
        nodes = list(self.graph.nodes) # need this copy so 'nodes' doesn't change size on iteration
        for node in nodes:
            if node in self.redirect_sources:
                self.graph.remove_node(node)
    
    def update_next_links(self, articles):
        for article in articles:
            for link in article['links']:
                self.next_links.append(link)

    def query_articles(self, titles, generate_graph=True):
        articles = wiki_multi_query(titles)
        
        self.update_redirects(articles)
        
        self.update_next_links(articles)
        self.add_edges(articles)

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [45]:
gc = GraphCreator("Decision tree")
len(gc.next_links)

88

2. We query all the nodes linked to from the entry point (expand our network one level for each node).

In [42]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [43]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [44]:
gc.update_edge_weights()

# Getting Our Feature Set

We have to options when generating our feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

In [32]:
features_df = gc.get_features_df(rank=True)
features_df

FULL


,node,degree,degree_ranked,category_matches_with_source,category_matches_with_source_ranked,in_edges,out_edges,centrality,centrality_ranked,dispersion,dispersion_ranked,page_rank,page_rank_ranked,adjusted_reciprocity,adjusted_reciprocity_ranked,shortest_path_length_from_source,shortest_path_length_from_source_ranked
0,Algorithm,3378,1,0,2,2990.0,388.0,9.241492e-02,829,0.857143,16.0,0.048183,1,204.0,18,1.0,2.0
1,Time,2899,2,0,2,1973.0,926.0,1.439158e-01,846,1.142857,20.0,0.040602,2,1086.0,1,1.0,2.0
2,Causality,2279,3,0,2,1280.0,999.0,1.317196e-01,843,1.666667,29.0,0.026763,4,962.0,3,1.0,2.0
3,Probability,1865,4,0,2,1571.0,294.0,5.033018e-02,804,0.750000,13.0,0.027178,3,238.0,16,1.0,2.0
4,Operations research,1856,5,0,2,1401.0,455.0,6.244140e-02,812,0.692308,9.0,0.021569,5,338.0,10,1.0,2.0
5,Semantic Web,1388,6,0,2,784.0,604.0,1.432438e-01,845,0.000000,1.0,0.016514,6,764.0,4,1.0,2.0
6,Flowchart,1290,7,0,2,815.0,475.0,2.391762e-01,853,0.166667,3.0,0.013175,9,722.0,5,1.0,2.0
7,Topic map,1231,8,0,2,626.0,605.0,2.864750e-01,854,2.466667,39.0,0.013776,8,1032.0,2,1.0,2.0
8,Utility,1222,9,0,2,746.0,476.0,1.825126e-02,735,0.250000,4.0,0.011688,10,470.0,7,1.0,2.0
9,Expected value,970,10,0,2,832.0,138.0,8.875428e-03,583,0.250000,4.0,0.015080,7,110.0,32,1.0,2.0


In [22]:
def average_rank(row):
    return np.mean([
        row.degree_ranked,
        row.centrality_ranked,
        row.dispersion_ranked,
        row.page_rank_ranked,
        row.adjusted_reciprocity_ranked,
    ]) * row.category_matches_with_source_ranked * row.shortest_path_length_from_source_ranked 

features_df["rank_average"] = features_df.apply(average_rank, axis=1)

features_df.sort_values("rank_average", ascending=True)

,node,degree,degree_ranked,category_matches_with_source,category_matches_with_source_ranked,in_edges,out_edges,centrality,centrality_ranked,dispersion,dispersion_ranked,page_rank,page_rank_ranked,adjusted_reciprocity,adjusted_reciprocity_ranked,shortest_path_length_from_source,shortest_path_length_from_source_ranked,rank_average
3,Trademark,2544,4,0,4,2304.0,240.0,1.470975e-03,255,0.333333,11.0,0.037949,3,224.0,23,1.0,2.0,473.6
100,Gradient boosting,137,90,3,1,25.0,112.0,1.596584e-02,1214,2.455556,44.0,0.000170,159,20.0,67,1.0,2.0,629.6
51,Random forest,288,51,1,3,161.0,127.0,1.185048e-01,1347,12.543307,96.0,0.000938,64,166.0,35,0.0,1.0,955.8
153,Donald Geman,55,112,0,4,14.0,41.0,1.327016e-03,217,2.111111,39.0,0.000115,256,6.0,76,1.0,2.0,1120.0
178,Random tree,33,127,0,4,10.0,23.0,1.343511e-03,222,0.000000,1.0,0.000096,286,10.0,73,1.0,2.0,1134.4
14,Statistical classification,1142,15,2,2,689.0,453.0,1.616147e-01,1361,2.736264,54.0,0.003886,18,652.0,5,1.0,2.0,1162.4
412,Partial permutation,23,136,0,4,7.0,16.0,1.209916e-03,191,1.000000,17.0,0.000085,312,2.0,78,1.0,2.0,1174.4
15,Artificial neural network,1139,16,2,2,815.0,324.0,1.198236e-01,1348,4.640449,80.0,0.008248,8,262.0,20,1.0,2.0,1177.6
40,Decision tree learning,326,40,2,2,172.0,154.0,9.166467e-02,1312,5.587629,89.0,0.000852,69,182.0,29,1.0,2.0,1231.2
75,Bootstrap aggregating,230,69,2,2,130.0,100.0,9.534763e-02,1316,1.977528,37.0,0.000641,87,150.0,43,1.0,2.0,1241.6


In [ ]:
# features_df.dispersion = features_df.dispersion.fillna(0.0)
# features_df.shortest_path_length_from_source = features_df.shortest_path_length_from_source.fillna(-1)

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())